[Пример 1](https://www.kaggle.com/code/arunlukedsouza/covid-19-chest-x-ray-classification-with-resnet-18)

[Пример 2](https://www.kaggle.com/code/arunrk7/covid-19-detection-pytorch-tutorial)

### Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd

import os

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision.datasets import ImageFolder

from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import Subset

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import random
from PIL import Image
from sklearn.metrics import confusion_matrix
import seaborn as sns

%matplotlib inline
torch.manual_seed(0)

### Проверка доступа к GPU

In [ ]:
if torch.cuda.is_available():
    device=torch.device("cuda:0")
    print("Training on GPU...")
else:
    device = torch.device("cpu")
    print("Training on CPU...")

### Преобразования (Transforms)

In [ ]:
# Creating a Transformation Object
train_transform = torchvision.transforms.Compose([
    # Converting images to the size that the model expects
    torchvision.transforms.Resize(size=(224, 224)),
    torchvision.transforms.RandomHorizontalFlip(), # A RandomHorizontalFlip to augment our data
    torchvision.transforms.ToTensor(), # Converting to tensor
    # Добавьте необходимую нормализацию,
    # если будете применять предобученную модель, например, ResNet18
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]) # Normalizing the data to the data that the ResNet18 was trained on
    
])


val_transform = torchvision.transforms.Compose([
    # Converting images to the size that the model expects
    torchvision.transforms.Resize(size=(224, 224)),
    torchvision.transforms.ToTensor(), # Converting to tensor
    # Добавьте необходимую нормализацию,
    # если будете применять предобученную модель, например, ResNet18
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]) # Normalizing the data to the data that the ResNet18 was trained on
    
])


test_transform = torchvision.transforms.Compose([
    # Converting images to the size that the model expects
    torchvision.transforms.Resize(size=(224, 224)),
    torchvision.transforms.ToTensor(), # Converting to tensor
    # Добавьте необходимую нормализацию,
    # если будете применять предобученную модель, например, ResNet18
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]) # Normalizing the data to the data that the ResNet18 was trained on
    
])

### Создание объектов Datasets с разбиением выборки на обучающую и валидационную

In [ ]:
train_val_path="/kaggle/input/radiograph-classification-2025/Dataset"

train_dataset = ImageFolder(train_val_path, transform=train_transform)
val_dataset = ImageFolder(train_val_path, transform=val_transform)

class_names = train_dataset.classes
print(class_names) # list out all the classes

In [ ]:
# Splitting the data into train and validation set
def split_train_val(tot_img, val_percentage=0.2, rnd=23):
    # Here indices are randomly permuted 
    number_of_val = int(tot_img*val_percentage)
    
    np.random.seed(rnd)
    indexs = np.random.permutation(tot_img)
    return indexs[0:number_of_val], indexs[number_of_val:]

randomness = 1
val_per = 0.2

all_len = len(train_dataset)

val_indices, train_indices = split_train_val(all_len, val_per, randomness)

print(val_indices, "validation data:", val_indices.shape)
print(train_indices, "train data:", train_indices.shape)

In [ ]:
train_dataset = Subset(train_dataset, train_indices)
val_dataset = Subset(val_dataset, val_indices)

In [ ]:
img0, label0 = train_dataset[9627]
print(img0.shape, label0)

img1,label1 = val_dataset[20]
print(img1.shape, label1)

In [ ]:
def show(img, label):
    print("label-->",class_names[label])
    img = img.numpy().transpose((1, 2, 0)) # Channel first then height and width
    # Если вы применили нормализацию, например, для ResNet18,
    # то для визуализации и корректного отображения нужно сделать обратные преобразования
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = img * std + mean
    img = np.clip(img, 0., 1.)
    plt.imshow(img)

show(*train_dataset[6])

### Dataloaders

In [ ]:
batch_size = 10

train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size, shuffle=False)

### Визуализация примеров

In [ ]:
def show_images(images, labels, preds):
    plt.figure(figsize=(10,10))
    
    images = images.cpu()
    labels = labels.cpu()
    preds = preds.cpu()
    
    for i, image in enumerate(images):
        plt.subplot(1, batch_size, i + 1, xticks = [], yticks =[]) # x & y ticks are set to blank
        image = image.numpy().transpose((1, 2, 0)) # Channel first then height and width
        
        # Если вы применили нормализацию, например, для ResNet18,
        # то для визуализации и корректного отображения нужно сделать обратные преобразования
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = image * std + mean
        image = np.clip(image, 0., 1.)
        plt.imshow(image)
        
        col = 'green' if preds[i] == labels[i] else 'red'
        
        plt.xlabel(f'{class_names[int(labels[i].numpy())]}')
        plt.ylabel(f'{class_names[int(preds[i].numpy())]}', color=col)
    plt.tight_layout()
    plt.show()

In [ ]:
images, labels = next(iter(train_dataloader)) # Fetch the next batch of images
show_images(images, labels, labels)

In [ ]:
images, labels = next(iter(val_dataloader))
show_images(images, labels, labels)

### Создание модели

In [ ]:
'''
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, 4)  # Output size: 4 classes

    def forward(self, x):
        # current x.shape [batch_size, 3, 224, 224]
        # First convolutional layer with ReLU and max pooling
        x = F.relu(self.conv1(x))
        # current x.shape [batch_size, 16, 224, 224]
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # Output size: (16, 112, 112)
        # current x.shape [batch_size, 16, 112, 112]

        # Second convolutional layer with ReLU and max pooling
        x = F.relu(self.conv2(x))
        # current x.shape [batch_size, 32, 112, 112]
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # Output size: (32, 56, 56)
        # current x.shape [batch_size, 32, 56, 56]

        # Third convolutional layer with ReLU and max pooling
        x = F.relu(self.conv3(x))
        # current x.shape [batch_size, 64, 56, 56]
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # Output size: (64, 28, 28)
        # current x.shape [batch_size, 64, 28, 28]

        # Flatten the tensor for the fully connected layer
        x = x.view(x.size(0), -1)  # Flatten the output to (batch_size, 64*28*28)
        # current x.shape [batch_size, 50176]

        # First fully connected layer
        x = F.relu(self.fc1(x))
        # current x.shape [batch_size, 128]
        
        # Output layer
        x = self.fc2(x)  # Output shape: (batch_size, 4)
        # current x.shape [batch_size, 4]
        return x
        

model = SimpleCNN()
print(model)
'''

In [ ]:
model_resnet18 = torchvision.models.resnet18(pretrained=True)

In [ ]:
class_names

In [ ]:
'''
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
'''

In [ ]:
model_resnet18.fc = torch.nn.Linear(in_features=512, out_features=4)
model_resnet18 = model_resnet18.to(device)

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adagrad(model_resnet18.parameters(), lr=39e-5)
#AdamW
#6e-4 accuracy сильно скачет вверх-вниз
#45e-5 accuracy сильно скачет вверх-вниз
#35e-5 более стабильно, но все равно accuracy сильно скачет вверх-вниз (0,92 - 0,77 - 0,86)

#Adagrad
#35e-5 довольно быстро достигаем accuracy 0.9-0.92 и колеблемся в этих пределах, доползли до 0.94, в целом неплохо
#31e-5 примерно так же, как в прошлый раз

In [ ]:
def show_preds():
    model_resnet18.eval()  # Setting the model to evaluation mode
    images, labels = next(iter(val_dataloader))
    
    with torch.no_grad():
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_resnet18(images)
        
    _, preds = torch.max(outputs, 1)
    show_images(images, labels, preds)

In [ ]:
show_preds()

In [ ]:
def train(epochs):
    print('Starting training..')
    
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    for e in range(0, epochs):
        print('='*20)
        print(f'Starting epoch {e + 1}/{epochs}')
        print('='*20)

        train_loss = 0.
        val_loss = 0.  # Not computing val_loss since we'll be evaluating the model multiple times within one epoch
        
        
        model_resnet18.train() # set model to training phase
        
        for train_step, (images, labels) in enumerate(train_dataloader):
            
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model_resnet18(images)
            loss = criterion(outputs, labels)
            # Once we get the loss we need to take a gradient step
            loss.backward() # Back propagation
            optimizer.step() # Completes the gradient step by updating all the parameter values (we are using all parameters)
            train_loss += loss.item() # Loss is a tensor which can't be added to train_loss so .item() converts it to float
            
            # Evaluating the model every 20th step
            if train_step % 20 == 0:
                print('Evaluating at step', train_step)

                accuracy = 0

                model_resnet18.eval() # set model to eval phase

                all_preds = []
                all_labels = []

                for val_step, (images, labels) in enumerate(val_dataloader):
                    
                    images = images.to(device)
                    labels = labels.to(device)
                    
                    with torch.no_grad():
                        outputs = model_resnet18(images)
                        
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

                    _, preds = torch.max(outputs, 1)
                    accuracy += sum((preds.cpu() == labels.cpu()).numpy()) # adding correct preds to acc

                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())


                val_loss /= (val_step + 1)
                accuracy = accuracy/len(val_dataset)
                print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')

                val_losses.append(val_loss)
                val_accuracies.append(accuracy)

                #show_preds()

                model_resnet18.train()

                if accuracy >= 0.95:
                    print('Performance condition satisfied, stopping..')
                    break

        train_loss /= (train_step + 1)
        train_losses.append(train_loss)

        print(f'Training Loss: {train_loss:.4f}')
    print('Training complete..')

    plt.figure(figsize=(12, 5))
    
    # Plot training and validation loss
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    
    # Plot validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(val_accuracies, label='Validation Accuracy', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Validation Accuracy')
    plt.legend()
    
    plt.show()

    # Confusion Matrix
    model_resnet18.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_resnet18(images)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

### Запуск обучения

In [ ]:
%%time

train(epochs=2)

### Создание класса TestDataset
Наследуемся от базового класса Dataset, модифицируем метод get_item, который теперь возвращает только изображение, т.к. метка класса неизвестна и ее нужно предсказать в рамках соревнования

In [ ]:
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None): 
        self.root_dir = root_dir
        self.transform = transform
        self.filenames = sorted(os.listdir(root_dir))

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.filenames[idx])
        
        image = Image.open(img_name)
        image = image.convert('RGB')
        
        if self.transform:
            sample = self.transform(image)

        return sample

In [ ]:
root_dir = '/kaggle/input/radiograph-classification-2025/Test'
test_dataset = TestDataset(root_dir, test_transform)

In [ ]:
img0 = test_dataset[112]
print(img0.shape)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)

### Predict на тестовой выборке

In [ ]:
# Generate predictions
predictions = []

model_resnet18.eval()  # Set model to evaluation mode


for images in test_dataloader:
    images = images.to(device)
    
    with torch.no_grad():
        outputs = model_resnet18(images)
        
    _, preds = torch.max(outputs, 1)
    
    preds = preds.cpu()
    predictions.extend(preds.tolist())

### Формирование файла submission

In [ ]:
predictions_df = pd.DataFrame({'ImageId': range(1, len(predictions) + 1), 'Label': predictions})

predictions_df

In [ ]:
predictions_df.to_csv('submission.csv', index = False)